# My portfolios

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import etfs as et

In [ ]:
from etfs.portfolio.io import import_portfolio, import_portfolio_vanguard, parse_portfolio, parse_portfolio_vanguard
from etfs.stats.basics import resample_df, runrate_column, shift_column, \
                              standard_deviation_column, difference, rsq, beta, alpha
from etfs.utils.helpers import todays_date
from etfs.visualization.trends import plot_composition

## RobinHood

In [ ]:
df_rh, rh = import_portfolio(path="../data/portfolio_rh.csv", name="RobinHood")

In [ ]:
rh.overview()

In [ ]:
rh.positions()

In [ ]:
rh.get_timeseries()

In [ ]:
rh.timeseries.plot(y=rh.timeseries.columns)

In [ ]:
plot_composition(portfolio=rh)

In [ ]:
rh.timeseries_growth.plot(y="Growth")

In [ ]:
# Volatility of portfolio
df_volatility = standard_deviation_column(rh.timeseries, column='Total', window=1, shift=1)

In [ ]:
rh.get_benchmark()

In [ ]:
rh.get_returns()

In [ ]:
total = et.TotalPortfolioValue(name="Total", data=rh.timeseries) # to-do: turn this into portfolio object

In [ ]:
df_volatility = standard_deviation_column(total.data, column='Total', window=1, shift=1)

In [ ]:
total.get_returns()

In [ ]:
total.get_benchmark()

In [ ]:
total.benchmark.get_returns()

In [ ]:
rsq(total, total.benchmark, col1='Return', col2='Return')

In [ ]:
beta(sec1=total, sec2=total.benchmark, col1='Return', col2='Return')

In [ ]:
alpha(sec1=total, sec2=total.benchmark, col1='Return', col2='Return', risk_free_rate=.004484)

## Vanguard

In [ ]:
df_vg, vg = import_portfolio_vanguard(path="../data/portfolio_vanguard.csv", name="Vanguard")

In [ ]:
vg.overview()     

In [ ]:
vg.positions()

In [ ]:
vg.get_timeseries()

In [ ]:
vg.timeseries.plot(y=vg.timeseries.columns)

In [ ]:
plot_composition(portfolio=vg)

In [ ]:
vg.timeseries_growth.plot(y="Growth")

In [ ]:
vg.get_benchmark()

In [ ]:
# Volatility of portfolio
df_volatility = standard_deviation_column(vg.timeseries, column='Total', window=1, shift=1)

In [ ]:
df_volatility = standard_deviation_column(vg.benchmark.data, column='Close', window=1, shift=1)

In [ ]:
total = et.TotalPortfolioValue(name="Total", data=vg.timeseries) # to-do: turn this into portfolio object

In [ ]:
total.get_benchmark()

In [ ]:
total.get_returns()

In [ ]:
rsq(total, total.benchmark, col1='Return', col2='Return')

In [ ]:
beta(sec1=total, sec2=total.benchmark, col1='Return', col2='Return')

In [ ]:
alpha(sec1=total, sec2=total.benchmark, col1='Return', col2='Return', risk_free_rate=.004484)

## Another Vanguard

In [ ]:
df_vg2, vg_2 = import_portfolio_vanguard(path="../data/portfolio_vanguard_2.csv", name="Vanguard")

In [ ]:
vg_2.overview()

In [ ]:
vg_2.positions()

In [ ]:
vg_2.get_timeseries()

In [ ]:
vg_2.timeseries.plot(y=vg_2.timeseries.columns)

In [ ]:
plot_composition(portfolio=vg_2)

In [ ]:
vg_2.timeseries_growth.plot(y="Growth")

In [ ]:
vg_2.get_benchmark()

In [ ]:
# Volatility of portfolio
df_volatility = standard_deviation_column(vg_2.timeseries, column='Total', window=1, shift=1)

In [ ]:
df_volatility = standard_deviation_column(vg_2.benchmark.data, column='Close', window=1, shift=1)

In [ ]:
total = et.TotalPortfolioValue(name="Total", data=vg_2.timeseries) # to-do: turn this into portfolio object

In [ ]:
total.get_benchmark()

In [ ]:
total.get_returns()

In [ ]:
rsq(total, total.benchmark, col1='Return', col2='Return')

In [ ]:
beta(sec1=total, sec2=total.benchmark, col1='Return', col2='Return')

In [ ]:
alpha(sec1=total, sec2=total.benchmark, col1='Return', col2='Return', risk_free_rate=.004484)

## Add up portfolios

In [ ]:
tot = et.Portfolio(name="tot")

In [ ]:
tot = parse_portfolio(df_rh, tot)

In [ ]:
tot = parse_portfolio_vanguard([df_vg, df_vg2], tot)

In [ ]:
tot.get_timeseries()

In [ ]:
tot.timeseries_growth.plot(y="Growth")

In [ ]:
tot.overview()

In [ ]:
tot.positions()

In [ ]:
plot_composition(portfolio=tot)

In [ ]:
total = et.TotalPortfolioValue(name="Total", data=tot.timeseries) # to-do: turn this into portfolio object

In [ ]:
total.get_returns()

In [ ]:
total.get_benchmark()

In [ ]:
rsq(sec1=total, sec2=total.benchmark, col1='Return', col2='Return')

In [ ]:
beta(sec1=total, sec2=total.benchmark, col1='Return', col2='Return')

In [ ]:
alpha(sec1=total, sec2=total.benchmark, col1='Return', col2='Return', risk_free_rate=.004484)

## Show correlation of positions

In [ ]:
import seaborn as sns
import numpy as np

In [ ]:
sns.set(style="white")

tot.tickers.sort()
# Compute the correlation matrix
corr = tot.timeseries[tot.tickers].corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

## Restrict to trading days

In [ ]:
import pandas_market_calendars as mcal

In [ ]:
tot.timeseries

In [ ]:
tot.min_date

In [ ]:
tot.max_date

In [ ]:
nyse = mcal.get_calendar('NYSE')

In [ ]:
schedule = nyse.schedule(start_date=tot.min_date, end_date=todays_date())

In [ ]:
tot.timeseries = tot.timeseries.join(schedule, how='inner')